In [4]:
# Load model directly
from transformers import AutoProcessor, AutoModelForImageTextToText, Florence2ForConditionalGeneration

processor = AutoProcessor.from_pretrained("microsoft/Florence-2-base", trust_remote_code=True)
model = Florence2ForConditionalGeneration.from_pretrained("microsoft/Florence-2-base", trust_remote_code=True)


Some weights of the model checkpoint at microsoft/Florence-2-base were not used when initializing Florence2ForConditionalGeneration: ['image_pos_embed.column_embeddings.weight', 'image_pos_embed.row_embeddings.weight', 'image_proj_norm.bias', 'image_proj_norm.weight', 'image_projection', 'language_model.final_logits_bias', 'language_model.model.decoder.embed_positions.weight', 'language_model.model.decoder.layernorm_embedding.bias', 'language_model.model.decoder.layernorm_embedding.weight', 'language_model.model.decoder.layers.0.encoder_attn.k_proj.bias', 'language_model.model.decoder.layers.0.encoder_attn.k_proj.weight', 'language_model.model.decoder.layers.0.encoder_attn.out_proj.bias', 'language_model.model.decoder.layers.0.encoder_attn.out_proj.weight', 'language_model.model.decoder.layers.0.encoder_attn.q_proj.bias', 'language_model.model.decoder.layers.0.encoder_attn.q_proj.weight', 'language_model.model.decoder.layers.0.encoder_attn.v_proj.bias', 'language_model.model.decoder.la

In [2]:
pip show transformers

Name: transformers
Version: 4.57.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.12/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


In [3]:
pip install transformers==4.40 #florence2 not supported for transformers >= 4.49. Requires 4.40 at least to run

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 55.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.2 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.0 which is incompatible.


In [5]:
#example code -- pipeline
# Use a pipeline as a high-level helper
import torch
from transformers import pipeline

pipe = pipeline("image-text-to-text", model="microsoft/Florence-2-base", trust_remote_code=True)

AttributeError: partially initialized module 'torch' has no attribute 'distributed' (most likely due to a circular import)

In [15]:
import transformers
from transformers import AutoProcessor, Florence2ForConditionalGeneration
from PIL import Image
import requests
transformers.logging.set_verbosity_error() #gets rid of error messages
# Load model directly

processor = AutoProcessor.from_pretrained("microsoft/Florence-2-base", trust_remote_code=True)
model = Florence2ForConditionalGeneration.from_pretrained("microsoft/Florence-2-base", trust_remote_code=True)

image = Image.open("R2D2_1.jpg")

inputs = processor(text="<ITS><image>Describe the image", images=image, return_tensors="pt")
outputs = model.generate(**inputs)
print(processor.decode(outputs[0]))


ValueError: Image features and image tokens do not match: tokens: 0, features 577

In [8]:
import transformers
from transformers import AutoProcessor, Florence2ForConditionalGeneration
from PIL import Image
import torch

transformers.logging.set_verbosity_error()

# Convert image to RGB and print mode/size
image = Image.open("R2D2_1.jpg")
print("Image mode / size:", image.mode, image.size)
if image.mode != "RGB":
    image = image.convert("RGB")
    print("Converted to RGB")

# load with trust_remote_code
processor = AutoProcessor.from_pretrained("microsoft/Florence-2-base", trust_remote_code=True)
model = Florence2ForConditionalGeneration.from_pretrained("microsoft/Florence-2-base", trust_remote_code=True)

# Use batch form (lists) — this often avoids token/feature mismatch
prompt = "<image><ITS>Describe the image."
texts = [prompt]
images = [image]

inputs = processor(text=texts, images=images, return_tensors="pt")

# Print returned keys and shapes / lengths
print("\nPROCESSOR OUTPUT KEYS:")
for k, v in inputs.items():
    try:
        print(f"  {k}: type={type(v)}, shape={getattr(v, 'shape', 'n/a')}, len(if tensor)={(v.shape if hasattr(v,'shape') else 'n/a')}")
    except Exception as e:
        print(f"  {k}: (error printing) {e}")

# If there is an 'input_ids' inspect tokens
if "input_ids" in inputs:
    print("\ninput_ids shape:", inputs["input_ids"].shape)
    # Show first 60 token ids and the decoded text (short)
    print("input_ids[0][:60]:", inputs["input_ids"][0][:60].tolist())
    try:
        print("decoded prompt:", processor.tokenizer.decode(inputs["input_ids"][0].tolist(), skip_special_tokens=False))
    except Exception as e:
        print("Could not decode input_ids:", e)

# If there is an 'image' or 'image_features' or 'pixel_values' key, show shape
for k in ("pixel_values", "image_features", "images", "image_embeds"):
    if k in inputs:
        print(f"{k} shape:", getattr(inputs[k], "shape", "n/a"))

# Try a quick forward pass to catch the same error and print stack
try:
    outputs = model.generate(**{k:v.to(model.device) if isinstance(v, torch.Tensor) else v for k,v in inputs.items()})
    print("\nGenerated (ids):", outputs)
    print("Decoded:", processor.decode(outputs[0]))
except Exception as e:
    import traceback
    print("\n--- MODEL RUN ERROR ---")
    traceback.print_exc()


Image mode / size: RGB (576, 484)

PROCESSOR OUTPUT KEYS:
  input_ids: type=<class 'torch.Tensor'>, shape=torch.Size([1, 12]), len(if tensor)=torch.Size([1, 12])
  attention_mask: type=<class 'torch.Tensor'>, shape=torch.Size([1, 12]), len(if tensor)=torch.Size([1, 12])
  pixel_values: type=<class 'torch.Tensor'>, shape=torch.Size([1, 3, 768, 768]), len(if tensor)=torch.Size([1, 3, 768, 768])

input_ids shape: torch.Size([1, 12])
input_ids[0][:60]: [0, 41552, 20094, 49007, 27510, 15698, 47066, 21700, 5, 2274, 4, 2]
decoded prompt: <s><image><ITS>Describe the image.</s>
pixel_values shape: torch.Size([1, 3, 768, 768])

--- MODEL RUN ERROR ---


Traceback (most recent call last):
  File "/tmp/ipython-input-1466727224.py", line 51, in <cell line: 0>
    outputs = model.generate(**{k:v.to(model.device) if isinstance(v, torch.Tensor) else v for k,v in inputs.items()})
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/_contextlib.py", line 120, in decorate_context
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/transformers/generation/utils.py", line 2465, in generate
    model_kwargs = self._prepare_encoder_decoder_kwargs_for_generation(
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/transformers/models/florence2/modeling_florence2.py", line 1036, in _prepare_encoder_decoder_kwargs_for_generation
    special_image_mask = self.get_placeholder_mask(
                   

In [11]:
pip install huggingface_hub

In [14]:
from huggingface_hub import scan_cache

cache_info = scan_cache()
print(cache_info)

ImportError: cannot import name 'scan_cache' from 'huggingface_hub' (/usr/local/lib/python3.12/dist-packages/huggingface_hub/__init__.py)

In [20]:
import transformers
from transformers import AutoProcessor, Florence2ForConditionalGeneration
from PIL import Image
import torch

processor = AutoProcessor.from_pretrained(
    "microsoft/Florence-2-base",
    trust_remote_code=True
)

processor.num_additional_image_tokens = 1

model = Florence2ForConditionalGeneration.from_pretrained(
    "microsoft/Florence-2-base",
    trust_remote_code=True
)

image = Image.open("R2D2_1.jpg")

prompt = "<ITS><image>Describe the image."

inputs = processor(
    text=prompt,
    images=image,
    return_tensors="pt"
).to(model.device)

outputs = model.generate(**inputs)

print(processor.decode(outputs[0], skip_special_tokens=True))


ValueError: Image features and image tokens do not match: tokens: 0, features 577

In [3]:
from transformers import AutoProcessor, Florence2ForConditionalGeneration # Reverted to correct model class
from PIL import Image
import torch

model_id = "microsoft/Florence-2-base"

processor = AutoProcessor.from_pretrained(
    model_id,
    trust_remote_code=True
)

# Reverted to Florence2ForConditionalGeneration
model = Florence2ForConditionalGeneration.from_pretrained(
    model_id,
    trust_remote_code=True
).to("cuda" if torch.cuda.is_available() else "cpu")

image = Image.open("R2D2_1.jpg")

prompt = "<CAPTION>"

# Removed the problematic image_token_id check, relying on processor to handle image tokens
inputs = processor(
    text=prompt,
    images=image,
    return_tensors="pt"
).to(model.device)

generated_ids = model.generate(
    **inputs,
    max_new_tokens=64
)

print(processor.decode(generated_ids[0], skip_special_tokens=True))


Some weights of the model checkpoint at microsoft/Florence-2-base were not used when initializing Florence2ForConditionalGeneration: ['image_pos_embed.column_embeddings.weight', 'image_pos_embed.row_embeddings.weight', 'image_proj_norm.bias', 'image_proj_norm.weight', 'image_projection', 'language_model.final_logits_bias', 'language_model.model.decoder.embed_positions.weight', 'language_model.model.decoder.layernorm_embedding.bias', 'language_model.model.decoder.layernorm_embedding.weight', 'language_model.model.decoder.layers.0.encoder_attn.k_proj.bias', 'language_model.model.decoder.layers.0.encoder_attn.k_proj.weight', 'language_model.model.decoder.layers.0.encoder_attn.out_proj.bias', 'language_model.model.decoder.layers.0.encoder_attn.out_proj.weight', 'language_model.model.decoder.layers.0.encoder_attn.q_proj.bias', 'language_model.model.decoder.layers.0.encoder_attn.q_proj.weight', 'language_model.model.decoder.layers.0.encoder_attn.v_proj.bias', 'language_model.model.decoder.la

ValueError: Image features and image tokens do not match: tokens: 0, features 577